In [1]:
import torch.nn.functional

from model import UNet
from lfw_dataset import LFWDataset
from torch.utils.data.dataloader import DataLoader
import numpy as np
from torchvision.transforms import v2
import cv2
import matplotlib.pyplot as plt
from utils import transform_generator, inv_transform
from copy import deepcopy
import pickle
from utils import eval
from train import train
import wandb

/home/georgerapeanu/anaconda3/envs/AI2/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/georgerapeanu/anaconda3/envs/AI2/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please

In [2]:
sweep_config = {
    'method': 'grid'
    }
metric = {
    'name': 'val_loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'NUM_EPOCHS': {
        'values': [10, 20]
    },
     'BATCH_SIZE': {
         'values': [32, 48, 64, 80, 96]
      },
    'INPUT_SHAPE': {
        'values': [(64, 64)]
    },
    'NUM_LAYERS': {
        'values': [1, 2]
    },
    'LR': {
        'values': [0.004, 0.008, 0.01, 0.018, 0.026, 0.034, 0.038]
      },
    'INTERMEDIARY_FILTERS': {
        'values': [8, 16, 32]
    }
}

# parameters_dict = {
#     'NUM_EPOCHS': {
#         'values': [10, 20]
#     },
#      'BATCH_SIZE': {
#         # integers between 32 and 256
#         # with evenly-distributed logarithms 
#         'distribution': 'q_log_uniform_values',
#         'q': 8,
#         'min': 32,
#         'max': 96,
#       },
#     'INPUT_SHAPE': {
#         'values': [(64, 64)]
#     },
#     'NUM_LAYERS': {
#         'values': [1, 2]
#     },
#     'LR': {
#         # a flat distribution between 0 and 0.1
#         'distribution': 'uniform',
#         'min': 0.004,
#         'max': 0.04
#       },
#     'INTERMEDIARY_FILTERS': {
#         'values': [8, 16, 32]
#     }
# }

sweep_config['parameters'] = parameters_dict

In [3]:
sweep_id = wandb.sweep(sweep_config, project="cvdl")

Create sweep with ID: 7gi9hjfn
Sweep URL: https://wandb.ai/georgerapeanu/cvdl/sweeps/7gi9hjfn


In [4]:
def run_wandb():
    with wandb.init():
        train(None, wandb.config)

In [ ]:
wandb.agent(sweep_id, run_wandb, project="cvdl")

wandb: Agent Starting Run: qcuwrdqn with config:
wandb: 	BATCH_SIZE: 32
wandb: 	INPUT_SHAPE: [64, 64]
wandb: 	INTERMEDIARY_FILTERS: 8
wandb: 	LR: 0.004
wandb: 	NUM_EPOCHS: 10
wandb: 	NUM_LAYERS: 1
wandb: Currently logged in as: georgerapeanu. Use `wandb login --relogin` to force relogin


Epoch [1/10], Train Loss: 0.6914, Validation Loss: 0.6736
Epoch [2/10], Train Loss: 0.5472, Validation Loss: 0.6081
Epoch [3/10], Train Loss: 0.4670, Validation Loss: 0.5735
Epoch [4/10], Train Loss: 0.4127, Validation Loss: 0.4064
Epoch [5/10], Train Loss: 0.3862, Validation Loss: 0.4205
Epoch [6/10], Train Loss: 0.3684, Validation Loss: 0.4016
Epoch [7/10], Train Loss: 0.3590, Validation Loss: 0.4595
Epoch [8/10], Train Loss: 0.3481, Validation Loss: 0.3682
Epoch [9/10], Train Loss: 0.3311, Validation Loss: 0.4179
Epoch [10/10], Train Loss: 0.3339, Validation Loss: 0.3633


fw_intersection_over_union,▁▂▃▇▇█▆█▆█
mean_intersection_over_union,▁▂▄▇▇█▆█▆█
mean_pixel_accuracy,▁▃▅▇██▅▇▅▇
train_loss,█▅▄▃▂▂▂▁▁▁
val_loss,█▇▆▂▂▂▃▁▂▁
fw_intersection_over_union,0.75372
mean_intersection_over_union,0.6125
mean_pixel_accuracy,0.70898
train_loss,0.33386
val_loss,0.36335


wandb: Agent Starting Run: jrf3ixdq with config:
wandb: 	BATCH_SIZE: 32
wandb: 	INPUT_SHAPE: [64, 64]
wandb: 	INTERMEDIARY_FILTERS: 8
wandb: 	LR: 0.004
wandb: 	NUM_EPOCHS: 10
wandb: 	NUM_LAYERS: 2


Epoch [1/10], Train Loss: 0.7199, Validation Loss: 0.4631
Epoch [2/10], Train Loss: 0.3732, Validation Loss: 0.3489
Epoch [3/10], Train Loss: 0.2898, Validation Loss: 0.2842
Epoch [4/10], Train Loss: 0.2521, Validation Loss: 0.2903
Epoch [5/10], Train Loss: 0.2416, Validation Loss: 0.2388
Epoch [6/10], Train Loss: 0.2201, Validation Loss: 0.2257
Epoch [7/10], Train Loss: 0.2076, Validation Loss: 0.2206
Epoch [8/10], Train Loss: 0.2011, Validation Loss: 0.2369
Epoch [9/10], Train Loss: 0.1905, Validation Loss: 0.2113
Epoch [10/10], Train Loss: 0.1846, Validation Loss: 0.2128


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: iryv7xqo with config:
wandb: 	BATCH_SIZE: 32
wandb: 	INPUT_SHAPE: [64, 64]
wandb: 	INTERMEDIARY_FILTERS: 8
wandb: 	LR: 0.004
wandb: 	NUM_EPOCHS: 20
wandb: 	NUM_LAYERS: 1


Epoch [1/20], Train Loss: 0.6903, Validation Loss: 0.5463
Epoch [2/20], Train Loss: 0.4842, Validation Loss: 0.4728
Epoch [3/20], Train Loss: 0.4345, Validation Loss: 0.4867
Epoch [4/20], Train Loss: 0.3943, Validation Loss: 0.3954
Epoch [5/20], Train Loss: 0.3895, Validation Loss: 0.3941
Epoch [6/20], Train Loss: 0.3635, Validation Loss: 0.3768
Epoch [7/20], Train Loss: 0.3614, Validation Loss: 0.3696
Epoch [8/20], Train Loss: 0.3476, Validation Loss: 0.3678
Epoch [9/20], Train Loss: 0.3375, Validation Loss: 0.3631
Epoch [10/20], Train Loss: 0.3349, Validation Loss: 0.4530
Epoch [11/20], Train Loss: 0.3265, Validation Loss: 0.3215
Epoch [12/20], Train Loss: 0.3187, Validation Loss: 0.3722
Epoch [13/20], Train Loss: 0.3166, Validation Loss: 0.3469
Epoch [14/20], Train Loss: 0.3086, Validation Loss: 0.3591
Epoch [15/20], Train Loss: 0.3086, Validation Loss: 0.3703
Epoch [16/20], Train Loss: 0.3018, Validation Loss: 0.3212
Epoch [17/20], Train Loss: 0.2969, Validation Loss: 0.3366
Epoch 

fw_intersection_over_union,▁▃▃▆▆▆▇▇▇▄█▇▇▇▆█▇███
mean_intersection_over_union,▁▄▅▆▆▇▇▇▆▅█▇▇▇▆█▆█▇▇
mean_pixel_accuracy,▁▄▆▆▅▆▇▇▅▅▇█▆▆▅█▅█▆▆
train_loss,█▄▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
val_loss,█▆▆▃▃▃▃▃▂▅▁▃▂▂▃▁▂▁▁▁
fw_intersection_over_union,0.77538
mean_intersection_over_union,0.6568
mean_pixel_accuracy,0.73342
train_loss,0.28485
val_loss,0.32456


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: nzhie1xj with config:
wandb: 	BATCH_SIZE: 32
wandb: 	INPUT_SHAPE: [64, 64]
wandb: 	INTERMEDIARY_FILTERS: 8
wandb: 	LR: 0.004
wandb: 	NUM_EPOCHS: 20
wandb: 	NUM_LAYERS: 2


Epoch [1/20], Train Loss: 0.6377, Validation Loss: 0.4227
Epoch [2/20], Train Loss: 0.3426, Validation Loss: 0.2981
Epoch [3/20], Train Loss: 0.2732, Validation Loss: 0.2892
Epoch [4/20], Train Loss: 0.2446, Validation Loss: 0.2342
Epoch [5/20], Train Loss: 0.2236, Validation Loss: 0.2282
Epoch [6/20], Train Loss: 0.2111, Validation Loss: 0.2276
Epoch [7/20], Train Loss: 0.2074, Validation Loss: 0.2150
Epoch [8/20], Train Loss: 0.1960, Validation Loss: 0.2411
Epoch [9/20], Train Loss: 0.1910, Validation Loss: 0.2324
Epoch [10/20], Train Loss: 0.1817, Validation Loss: 0.1969
Epoch [11/20], Train Loss: 0.1746, Validation Loss: 0.2045
Epoch [12/20], Train Loss: 0.1734, Validation Loss: 0.1991
Epoch [13/20], Train Loss: 0.1659, Validation Loss: 0.2118


In [2]:
wandb.init(
    # set the wandb project where this run will be logged
    project="cvdl",
    
    # track hyperparameters and run metadata
    config={
        'NUM_EPOCHS': 10,
        'BATCH_SIZE': 32,
        'INPUT_SHAPE': (64, 64),
        'NUM_LAYERS': 1,
        'LR': 0.0713
    }
)


wandb: Currently logged in as: georgerapeanu. Use `wandb login --relogin` to force relogin


In [3]:
wandb.config


{'NUM_EPOCHS': 10, 'BATCH_SIZE': 32, 'INPUT_SHAPE': [64, 64], 'NUM_LAYERS': 1, 'LR': 0.0713}

In [4]:
NUM_EPOCHS = 100
BATCH_SIZE= 128
INPUT_SHAPE = (64, 64)
NUM_LAYERS = 2
LR = 0.01

ARTIFACTS_PATH='./artifacts'
BASE_PATH="./lfw_dataset"

In [5]:
train(None, wandb.config)

Epoch [1/10], Train Loss: 0.7203, Validation Loss: 1.6032
Epoch [2/10], Train Loss: 0.5678, Validation Loss: 0.9221
Epoch [3/10], Train Loss: 0.5055, Validation Loss: 0.5436
Epoch [4/10], Train Loss: 0.4451, Validation Loss: 0.5015
Epoch [5/10], Train Loss: 0.4202, Validation Loss: 0.5892
Epoch [6/10], Train Loss: 0.3807, Validation Loss: 0.5481
Epoch [7/10], Train Loss: 0.3618, Validation Loss: 0.4245
Epoch [8/10], Train Loss: 0.3465, Validation Loss: 0.3509
Epoch [9/10], Train Loss: 0.3230, Validation Loss: 0.3318
Epoch [10/10], Train Loss: 0.3040, Validation Loss: 0.3253


(UNet(
   (in_conv): DoubleConv(
     (double_conv): Sequential(
       (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
       (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU(inplace=True)
       (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
       (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (5): ReLU(inplace=True)
     )
   )
   (encoders): ModuleList(
     (0): EncoderBlock(
       (encoder): Sequential(
         (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
         (1): DoubleConv(
           (double_conv): Sequential(
             (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
             (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
             (2): ReLU(inplace=True)
             (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
             (4): BatchNorm2d(128, eps=1e-05,

In [6]:
wandb.finish()

fw_intersection_over_union,▁▁▅▆▄▅▇███
mean_intersection_over_union,▁▂▄▆▄▄▆███
mean_pixel_accuracy,▁▃▅▆▃▃▆▇██
train_loss,█▅▄▃▃▂▂▂▁▁
val_loss,█▄▂▂▂▂▂▁▁▁
fw_intersection_over_union,0.78398
mean_intersection_over_union,0.67025
mean_pixel_accuracy,0.78504
train_loss,0.30395
val_loss,0.32528


In [7]:
ds = LFWDataset(BASE_PATH, transforms=transform_generator(INPUT_SHAPE), split_name='test', download=False)
model.eval()

X, y = ds[2]
model_y = model(X.view(-1, *X.shape))
model_y = torch.nn.functional.interpolate(model_y, size=tuple(y.shape))
model_y = model_y.view(-1, *y.shape).argmax(dim=0)

_, model_y = inv_transform(X, model_y)
X, y = inv_transform(X, y)

print(X.shape, y.shape, model_y.shape)
fig, axes = plt.subplots(1, 3, figsize=(10, 5))

axes[0].imshow(X, cmap='gray')
axes[0].set_title('Input')

axes[1].imshow(cv2.cvtColor(model_y, cv2.COLOR_BGR2RGB))
axes[1].set_title('Output')

axes[2].imshow(cv2.cvtColor(y, cv2.COLOR_BGR2RGB))
axes[2].set_title('Ground truth')

# Remove ticks and labels for a cleaner display
for ax in axes:
    ax.axis('off')

# Show the plot
plt.show()

KeyboardInterrupt: 

In [ ]:
eval(model, LFWDataset(BASE_PATH, transforms=transform_generator(INPUT_SHAPE), download=False, split_name='test'))

In [ ]:
ds = LFWDataset(BASE_PATH, transforms=transform_generator(INPUT_SHAPE), split_name='validation', download=False)

In [ ]:
len(ds)

In [ ]:
ds[2][1].shape